# Sentiment analysis on GAP, Banana Republic and Old Navy by scraping articles from NYTimes using Azure API

ARTICLES RELATED TO GAP, BANANA REPUBLIC AND OLD NAVY ON NYTIMES IN THE PERIODS:  2013-2016  AND  2017-2020

In [1]:
import requests
import json
import urllib.request, urllib.error, urllib.parse
import pprint

In [2]:
#nytimes API Key
api_key = 'DdOe1d9QfwsW6ndHJ7yRUkK7gkWz7b3o'

In [3]:

def get_articles(query, api_key, page = 0, filter = ''):
    url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?query={}&page={}&api-key={}".format(query, page, api_key) + filter
    response = requests.get(url)
    return response.json()

In [4]:
#extracing article titles from NYTimes. 
articles = get_articles("\"GAP\"", api_key)

print(articles.keys())
print()
print(articles['status'])
print(articles['copyright'])

dict_keys(['status', 'copyright', 'response'])

OK
Copyright (c) 2023 The New York Times Company. All Rights Reserved.


In [5]:
print(articles['response'].keys())
print(articles['response']['meta'])

dict_keys(['docs', 'meta'])
{'hits': 94615, 'offset': 0, 'time': 107}


In [6]:
docs = articles['response']['docs'] # This has the articles
len(docs)

10

In [7]:
docs = []
import time

for i in range(10): # For 100 articles
    print(i)
    articles = get_articles("\"GAP\"", api_key, page = i, filter='&facet=true&begin_date=20170101&end_date=20201231')
    x = articles['response']['docs']
    docs.extend(x)
    time.sleep(12)


0
1
2
3
4
5
6
7
8
9


In [8]:
text = []
for doc in docs:
    text.append(doc['headline']['main'] + '. ' +  doc['abstract'])

In [9]:
#DEPLOYING AZURE API FOR TEXT ANALYTICS
import requests
import azure
import time

key = 'c4a9f61db2ef49329fd3545a3fee94ca' #"<paste-your-text-analytics-key-here>"
endpoint = 'https://wdazure.cognitiveservices.azure.com/' #"<paste-your-text-analytics-endpoint-here>"


from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential


def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client


def sentiment_analysis_example(client, txt):
    documents = [txt]
    response = client.analyze_sentiment(documents=documents)[0]
    print(response)
    responses_all.append(response)

def key_phrase_extraction_example(client, txt):

    try:
        documents = [txt]

        response = client.extract_key_phrases(documents = documents)[0]

        if not response.is_error:
            return response.key_phrases
        else:
            print(response.id, response.error)

    except Exception as err:
        print("Encountered exception. {}".format(err))

azure_keyphrases = []
azure_sentiments = []
responses_all =[]
txt_number = 0
requests_per_second_limit = 100
requests_per_minute_limit = 300
time_counter = time.time()  # Start time of processing the request
for txt in text:
    txt_number+=1
    print(txt_number)
    client = authenticate_client()
 # Detect key phrases.
    key_phrases = key_phrase_extraction_example(client,txt)
    azure_keyphrases.append(key_phrases)
 # Detect Sentiments
    sentiments = sentiment_analysis_example(client, txt)

    end_time = time.time()  # End time of processing the request
    processing_time = end_time - time_counter

    requests_in_this_second = txt_number % requests_per_second_limit
    requests_in_this_minute = txt_number % requests_per_minute_limit

    # If we reach the limit of requests per second, add a delay to maintain the limit
    if requests_in_this_second == 0:
        time_taken_for_batch = end_time - time_counter
        if time_taken_for_batch < 1:
            time.sleep(1 - time_taken_for_batch)
        time_counter = time.time()
    
    if requests_in_this_minute == 0:
        time_taken_for_minute = end_time - time_counter
        if time_taken_for_minute < 60:
            time.sleep(60 - time_taken_for_minute)
        time_counter = time.time()


1
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.08, neutral=0.43, negative=0.5), 'sentences': [SentenceSentiment(text=Gap widely mocked for tweet depicting half-blue, half-red sweatshirt.. , sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.08, neutral=0.43, negative=0.5), length=71, offset=0, mined_opinions=[]), SentenceSentiment(text=Gap, which has since deleted the tweet, said the sweatshirt was not actually a product for sale., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.02, neutral=0.55, negative=0.44), length=96, offset=71, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
2
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.11, neutral=0.85, negative=0.04), 'sentences': [SentenceSentiment(text=What Is the Future of Black Appalachia?. , sentiment

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.09, neutral=0.72, negative=0.19), 'sentences': [SentenceSentiment(text=America’s Death Gap. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.08, neutral=0.6, negative=0.32), length=21, offset=0, mined_opinions=[]), SentenceSentiment(text=And what else you need to know today., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.11, neutral=0.83, negative=0.06), length=37, offset=21, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
13
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.04, neutral=0.45, negative=0.52), 'sentences': [SentenceSentiment(text=‘Hope Gap’ Review: A Thin Line Between Love and War. Annette Bening and Bill Nighy play a couple that has lost its way after 29 years of marriage., sentiment=nega

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.39, negative=0.59), 'sentences': [SentenceSentiment(text=Gap Plans to Reopen Up to 800 Stores by the End of May. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.04, neutral=0.94, negative=0.01), length=56, offset=0, mined_opinions=[]), SentenceSentiment(text=The parent company of Gap, Old Navy, Banana Republic and Athleta said it aimed to be “responsibly aggressive” as it returned to business., sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.39, negative=0.59), length=137, offset=56, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
24
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.03, neutral=0.91, negative=0.06), 'sentences': [SentenceSentiment(text=Gender Gap Persists at L

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.18, neutral=0.74, negative=0.08), 'sentences': [SentenceSentiment(text=A Para-Swimmer Is Using the Gap Year to Dominate Cycling. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.01), length=58, offset=0, mined_opinions=[]), SentenceSentiment(text=Roderick Sewell, a master of adjustments, had barely used a handcycle before last year. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.08, neutral=0.72, negative=0.2), length=88, offset=58, mined_opinions=[]), SentenceSentiment(text=Now he’s hoping to become one of the few two-sport athletes in the Tokyo Games., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.46, neutral=0.51, negative=0.04), length=79, offset=146, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
36
{'id': '0', 'sentimen

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.08, neutral=0.85, negative=0.07), 'sentences': [SentenceSentiment(text=You Don’t Have to Be College-Bound to Take a Gap Year. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.11, neutral=0.69, negative=0.19), length=55, offset=0, mined_opinions=[]), SentenceSentiment(text=Want to save elephants? , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.23, neutral=0.72, negative=0.05), length=24, offset=55, mined_opinions=[]), SentenceSentiment(text=Try stonemasonry? , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.03, neutral=0.94, negative=0.03), length=18, offset=79, mined_opinions=[]), SentenceSentiment(text=Return to school? , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.03, neutral=0.94, negative=0.03), length=18, offset=97, mined_opinions=[]), Sentence

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.88, negative=0.1), 'sentences': [SentenceSentiment(text=Security Gap Leaves 885 Million Mortgage Documents Exposed. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.03, neutral=0.8, negative=0.17), length=60, offset=0, mined_opinions=[]), SentenceSentiment(text=The data, collected by First American Financial Corporation, included hundreds of millions of detailed and personal financial records., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.96, negative=0.03), length=134, offset=60, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
58
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.06, neutral=0.85, negative=0.09), 'sentences': [SentenceSentiment(text=Britain’s Equal Pay Day Hig

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.04, neutral=0.34, negative=0.63), 'sentences': [SentenceSentiment(text=Kamala Harris Announces Plan to Close the Gender Pay Gap. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.03, neutral=0.9, negative=0.07), length=58, offset=0, mined_opinions=[]), SentenceSentiment(text=Ms. Harris would force companies with 100 or more employees to show they were paying men and women equally for analogous work, penalizing those that didn’t., sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.04, neutral=0.34, negative=0.63), length=156, offset=58, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
68
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.04, neutral=0.84, negative=0.12), 'sentences': [SentenceSentiment(text=Brid

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.22, negative=0.76), 'sentences': [SentenceSentiment(text=The Gaps Between White and Black America, in Charts. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.99, negative=0.01), length=53, offset=0, mined_opinions=[]), SentenceSentiment(text=Racism is at the heart of our nation’s inequality., sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.02, neutral=0.22, negative=0.76), length=50, offset=53, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
78
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.23, neutral=0.73, negative=0.05), 'sentences': [SentenceSentiment(text=China’s Economy Grows, and Its Trade Gap With the U.S. Widens. , sentiment=neutral, confidence_scores=SentimentConf

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.3, negative=0.68), 'sentences': [SentenceSentiment(text=Turkish Attempt to Close Gap With the West Seems to Widen It. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.04, neutral=0.84, negative=0.12), length=62, offset=0, mined_opinions=[]), SentenceSentiment(text=The promise of an interview with the president of Turkey lured American reporters to Ankara, but that meeting apparently was never part of the plan., sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.3, negative=0.68), length=148, offset=62, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
89
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.03, neutral=0.13, negative=0.84), 'sentences': [SentenceSentiment(text=Tax the 

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.13, neutral=0.8, negative=0.06), 'sentences': [SentenceSentiment(text=Readers Tell Us: Is a Gap Year Worth It?. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.12, neutral=0.75, negative=0.13), length=42, offset=0, mined_opinions=[]), SentenceSentiment(text=The New York Times asked readers who took a year off from college to share their experiences. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.03, neutral=0.94, negative=0.03), length=94, offset=42, mined_opinions=[]), SentenceSentiment(text=Here’s what they said., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.26, neutral=0.71, negative=0.03), length=22, offset=136, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
99
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_sco

In [10]:
azure_keyphrases

[['half-blue, half-red sweatshirt', 'Gap', 'tweet', 'product', 'sale'],
 ['Black Appalachia',
  'cultural center',
  'rural Virginia',
  'Future',
  'curators',
  'efforts'],
 ['School Item',
  'Old Navy',
  'Child psychologists',
  'positive step',
  'colorful masks',
  'Children',
  'Crayola',
  'Disney',
  'brands',
  'normalcy'],
 ['The Real Lessons',
  'Yeezy Gap',
  'Telfar Clemens',
  'Telfar.',
  'Kanye',
  'outcry',
  'everyone'],
 ['Gap Strike 10-Year Deal',
  'Yeezy Gap’ Apparel Line',
  'Kanye West',
  'mall brand',
  'first half',
  'stores'],
 ['good, honest labor', 'Tom McCoy', 'Gap', 'puzzle'],
 ['Many employers',
  'college graduates',
  'Skills Gap',
  'Innovation',
  'Colleges',
  'job',
  'problem'],
 ['The Mensch Gap',
  'A mensch',
  'upstanding person',
  'NYT',
  'responsibility',
  'actions',
  'people',
  'America'],
 ['Gap Year', 'home'],
 ['The Complaint Gap', 'O.K.', 'NYT', 'girls'],
 ['Pay Gap', 'gender editor', 'The Times', 'Womansplaining', 'details'],
 

In [11]:
responses_all

[AnalyzeSentimentResult(id=0, sentiment=negative, warnings=[], statistics=None, confidence_scores=SentimentConfidenceScores(positive=0.08, neutral=0.43, negative=0.5), sentences=[SentenceSentiment(text=Gap widely mocked for tweet depicting half-blue, half-red sweatshirt.. , sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.08, neutral=0.43, negative=0.5), length=71, offset=0, mined_opinions=[]), SentenceSentiment(text=Gap, which has since deleted the tweet, said the sweatshirt was not actually a product for sale., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.02, neutral=0.55, negative=0.44), length=96, offset=71, mined_opinions=[])], is_error=False, kind=SentimentAnalysis),
 AnalyzeSentimentResult(id=0, sentiment=neutral, warnings=[], statistics=None, confidence_scores=SentimentConfidenceScores(positive=0.11, neutral=0.85, negative=0.04), sentences=[SentenceSentiment(text=What Is the Future of Black Appalachia?. , sentiment=neutral,

In [12]:
final_sentiments=[]
for i in responses_all:
    x = i['sentiment']
    final_sentiments.append(x)

In [13]:
final_sentiments

['negative',
 'neutral',
 'positive',
 'negative',
 'neutral',
 'positive',
 'negative',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'mixed',
 'neutral',
 'positive',
 'negative',
 'mixed',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 

In [14]:
final_sentiments.count('negative')

35

In [15]:
final_sentiments.count('positive')

7

In [16]:
final_sentiments.count('neutral')

56

# The above code has been repeated for Banana Republic and Old Navy